In [54]:
!git clone https://github.com/franciscogarate/utils_ucm

fatal: destination path 'utils_ucm' already exists and is not an empty directory.


In [4]:
import pandas as pd
import numpy as np
from utils_ucm import lx_unisex

In [55]:
lx_unisex

,LX
0,100000.0000
1,99002.7280
2,98002.9236
3,97003.0858
4,96003.2180
...,...
107,614.7622
108,470.9027
109,380.1981
110,331.2711


In [56]:
def lx(edad):
  return lx_unisex['LX'][edad]

lx(4)

96003.218

In [57]:
edad = 45
df = pd.DataFrame(pd.date_range(start='2024-12-31',periods=len(lx_unisex)-edad,freq='YE'), columns=['Fecha'])
df.head(5)

,Fecha
0,2024-12-31
1,2025-12-31
2,2026-12-31
3,2027-12-31
4,2028-12-31


In [58]:
df['edad'] = edad + df.index
df.tail(5)

,Fecha,edad
62,2086-12-31,107
63,2087-12-31,108
64,2088-12-31,109
65,2089-12-31,110
66,2090-12-31,111


In [59]:
df['t'] = df.index
df['lx'] = df['edad'].apply(lambda x: lx(x))
df.head(5)

,Fecha,edad,t,lx
0,2024-12-31,45,0,55015.6972
1,2025-12-31,46,1,54016.8787
2,2026-12-31,47,2,53018.2543
3,2027-12-31,48,3,52019.8557
4,2028-12-31,49,4,51021.7100


In [60]:
df['qx'] = df['lx'].diff(-1).fillna(0)/df['lx'][0]
df.head(5)

,Fecha,edad,t,lx,qx
0,2024-12-31,45,0,55015.6972,0.018155
1,2025-12-31,46,1,54016.8787,0.018152
2,2026-12-31,47,2,53018.2543,0.018148
3,2027-12-31,48,3,52019.8557,0.018143
4,2028-12-31,49,4,51021.7100,0.018138


In [61]:
def incr_capital(capital, t):
  return capital * (1 + 0.015) ** t

df['capital'] = incr_capital(5000, df.t)
df.head(5)

,Fecha,edad,t,lx,qx,capital
0,2024-12-31,45,0,55015.6972,0.018155,5000.000000
1,2025-12-31,46,1,54016.8787,0.018152,5075.000000
2,2026-12-31,47,2,53018.2543,0.018148,5151.125000
3,2027-12-31,48,3,52019.8557,0.018143,5228.391875
4,2028-12-31,49,4,51021.7100,0.018138,5306.817753


In [62]:
df['pagos'] = df['capital'] * df['qx']
df.head(5)

,Fecha,edad,t,lx,qx,capital,pagos
0,2024-12-31,45,0,55015.6972,0.018155,5000.000000,90.775774
1,2025-12-31,46,1,54016.8787,0.018152,5075.000000,92.119506
2,2026-12-31,47,2,53018.2543,0.018148,5151.125000,93.480157
3,2027-12-31,48,3,52019.8557,0.018143,5228.391875,94.858325
4,2028-12-31,49,4,51021.7100,0.018138,5306.817753,96.253737


In [63]:
df['qx'].sum()

1.0

In [64]:
i = 2.18/100
df['factor_desc'] = df['t'].apply(lambda t : 1 / (1 + i) ** t )
df.head(2)

,Fecha,edad,t,lx,qx,capital,pagos,factor_desc
0,2024-12-31,45,0,55015.6972,0.018155,5000.0,90.775774,1.000000
1,2025-12-31,46,1,54016.8787,0.018152,5075.0,92.119506,0.978665


In [65]:
df.pagos @ df.factor_desc

4178.944238269067

### Chequeo descuento lineal

In [66]:
!pip install numpy_financial

In [67]:
import numpy_financial as npf
npf.npv(i, df.pagos)

4178.944238269067

## Incorporando la tabla PASEM2020_Decesos_M_1ord

In [68]:
!pip install pyliferisk

In [53]:
from pyliferisk import MortalityTable, lx
from pyliferisk.mortalitytables import PASEM2020_Decesos_M_1ord
mt = MortalityTable(qx=PASEM2020_Decesos_M_1ord)

del df

df = pd.DataFrame(pd.date_range(start='2024-12-31',periods=(mt.w -edad),freq='YE'), columns=['Fecha'])
df['edad'] = edad + df.index
df['t'] = df.index
df['lx'] = df['edad'].apply(lambda x: mt.lx[x+1] if x <= mt.w else 0)
df.head(5)

,Fecha,edad,t,lx
0,2024-12-31,45,0,97835.314413
1,2025-12-31,46,1,97674.853474
2,2026-12-31,47,2,97486.724833
3,2027-12-31,48,3,97266.323352
4,2028-12-31,49,4,97009.819387
